In [ ]:
import os
import cv2
import glob
import shutil
import random
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from ultralytics import YOLO

result_img_width = 428
result_img_height = 240



In [ ]:
def process_and_merge_images(jpg_filename, png_filename):    
    jpg_image = Image.open(jpg_filename)
    png_image = Image.open(png_filename).convert("RGBA")
    
    # Change white to red in the PNG
    datas = png_image.getdata()
    new_data = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            new_data.append((255, 0, 0, item[3]))  # Change white to red
        elif item[0] == 0 and item[1] == 0 and item[2] == 0:
            new_data.append((255, 255, 255, 0))  # Set black to transparent
        else:
            new_data.append(item)
    png_image.putdata(new_data)
    
    # Resize PNG to match the JPG dimensions if necessary
    png_image = png_image.resize(jpg_image.size)
    
    # Composite images
    combined_image = Image.alpha_composite(jpg_image.convert("RGBA"), png_image)

    return combined_image

In [ ]:
def show_image(name):
    img1 = cv2.imread(f"./Dataset/Private_test_data/{name}.jpg")
    img1 = img1[:,:]

    img2 = cv2.imread(f"./Dataset/Private_inference/{name}.png")
    img2 = img2[:,:]

    plt.figure(facecolor='gray', figsize=(10, 10))
    plt.tight_layout()
    plt.subplot(221)
    plt.axis('off')
    plt.imshow(img1)
    plt.subplot(222)
    plt.axis('off')
    plt.imshow(img2)

    img3 = process_and_merge_images(f"./Dataset/Private_test_data/{name}.jpg", f"./Dataset/Private_inference/{name}.png")
    plt.subplot(223)
    plt.axis('off')
    plt.imshow(img3)


In [ ]:
def inference(best_model_path, test_image_path, target_image_path):
    best_model = YOLO(best_model_path)
    border_size = 5
    kernel = np.ones((10,10), np.uint8)

    for image_path in glob.glob(f"{test_image_path}*"):
        img = cv2.imread(image_path)
        black_img = cv2.imread("black.png", cv2.COLOR_BGR2GRAY)
        
        border_img = cv2.copyMakeBorder(img, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT, value=[0])
        results = best_model.predict(border_img, imgsz=640)

        if results[0].masks is None:
            results = best_model.predict(img, imgsz=640)

        for mask_xys in results[0].masks.xy:
            mask_xys = np.array(mask_xys).astype(np.int32)
            for xy in mask_xys:
                xy[0] -= border_size
                xy[1] -= border_size
                if xy[0] <= 3:
                    xy[0] = 2
                if xy[1] <= 2:
                    xy[1] = 2
            cv2.fillPoly(black_img, [mask_xys], color=(255,255,255))
        black_img = cv2.dilate(black_img, kernel)
        black_img = cv2.erode(black_img, kernel)
        
                
        black_img = cv2.copyMakeBorder(black_img, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT, value=[0])
        black_img = cv2.cvtColor(black_img, cv2.COLOR_BGR2GRAY)
        
        contours, _  = cv2.findContours(black_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0] - [border_size, border_size]

        adjusted_contours = []
        for contour in contours:
            contour = contour.squeeze(0)
            if contour[0] <= 3:
                contour[0] = 1
            if contour[1] <= 3:
                contour[1] = 1
            adjusted_contours.append(contour)

        black_img = cv2.imread("black.png", cv2.COLOR_BGR2GRAY)
        black_img = cv2.polylines(black_img, np.int32([adjusted_contours]), isClosed = True, color = (255, 255, 255), thickness = 2)
       
        black_img = cv2.resize(black_img, (result_img_width, result_img_height), interpolation=cv2.INTER_LINEAR)
        print(f"{target_image_path}{os.path.basename(image_path).replace('.jpg', '.png')}")
        cv2.imwrite(f"{target_image_path}{os.path.basename(image_path).replace('.jpg', '.png')}", black_img)
        

In [ ]:
best_model_path = "./runs/segment/train10/weights/best.pt"

public_test_image_path = "./Dataset/Public_test_data/"
private_test_image_path = "./Dataset/Private_test_data/"

os.makedirs("./Dataset/Public_inference/", exist_ok=True)
os.makedirs("./Dataset/Private_inference/", exist_ok=True)

target_public_test_image_path = "./Dataset/Public_inference/"
target_private_test_image_path = "./Dataset/Private_inference/"


inference(best_model_path, public_test_image_path, target_public_test_image_path)
inference(best_model_path, private_test_image_path, target_private_test_image_path)


In [ ]:
os.makedirs("./Dataset/Inference/", exist_ok=True)
for image_path in glob.glob("./Dataset/Public_inference/*"):
    shutil.copyfile(image_path, image_path.replace("Public_inference", "Inference"))
for image_path in glob.glob("./Dataset/Private_inference/*"):
    shutil.copyfile(image_path, image_path.replace("Private_inference", "Inference"))  
shutil.make_archive("./Dataset/zipdata/Inference", 'zip', "./Dataset/Inference/")

In [ ]:
image_path_list = []
for image_path in glob.glob("./Dataset/Private_inference/*"):
    image_path_list.append(os.path.basename(image_path)[:-4])

image_name = image_path_list[random.randint(0, len(image_path_list))]
show_image(image_name)